<a href="https://colab.research.google.com/github/MEGNEOrnela/NLP_week1/blob/main/MEGNE_CHOUDJA_Pauline_Ornela_n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [3]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [4]:

print("load training set..")
print("\n")
train_data, vocab = load_data("/content/drive/MyDrive/session3/train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("/content/drive/MyDrive/session3/valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [5]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []

    ## FILL CODE
    for words in data:
      word_lst=[]
      for word in words:
        if word in vocab.keys() and vocab[word]>mincount:
          word_lst.append(word)
        else:
          word_lst.append("<unk>")
        data_with_unk.append(word_lst)
    
    return data_with_unk

In [6]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
# train_data
print(train_data[0])

remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [7]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    seq_prev_counts=defaultdict(lambda: defaultdict(lambda: 0.0))
    
    for sentence in data:
        sentence = tuple(sentence)
        # print(sentence)

        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value

        for i in range(len(sentence)):
          total_number_words +=1
          for j in range(n):
            if i-j >= 0:
              counts[sentence[i-j:i]][sentence[i]] +=1


    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # # Build the probabilities from the counts
    # # Be careful with how you normalize!

    for context in counts.keys():
    # # ## FILL CODE
      denominator = 0
      for word in counts[context].keys():
          denominator += counts[context][word]
      for word in counts[context].keys():
          prob[context][word] = counts[context][word]/denominator 

    return prob

In [8]:
# RUN TO BUILD NGRAM MODEL

n = 2
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  2


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [9]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE
    if context in model:
        if w in model[context]:
          return model[context][w]
        else:
           return 0.4*get_prob(model,context[1:],w)
    else:
          return 0.4*get_prob(model,context[1:],w)
          

In [10]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE
    p, T =0.0, 0
    for sentence in data:
      sentence = tuple(sentence)
      for i in range(1,len(sentence)):
        if n-1<=i:
          context=sentence[i-(n-1):i]
          prob = get_prob(model, context, sentence[i])
          p+= - np.log(prob)
          T+=1

    perp = np.exp(p/T)

    return perp

In [11]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=2))

The perplexity is 68.72889287899149


In [12]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    if context in model:
        return model[context]
    else:
        return get_proba_distrib(model, context[1:])

    # return None

In [13]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE
        prob = get_proba_distrib(model, tuple(sentence))
        word = np.random.choice((list(prob.keys())), 1, p = list(prob.values()))
        sentence.append(word[0])
    return sentence

In [197]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'mary', 'caught', 'up', 'the', "teacher's", 'questions.', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.

## Try with larger training set (train2.txt and valid2.txt)

In [14]:
print("load training set..")
print("\n")
train_data2, vocab2 = load_data("/content/drive/MyDrive/session3/train2.txt")
print(train_data[0])
print("\n")
print("how :",vocab2['how'])
print("load validation set")
valid_data2, _ = load_data("/content/drive/MyDrive/session3/valid2.txt")


load training set..


['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


how : 3195
load validation set


In [15]:
print("remove rare words")
train_data2 = remove_rare_words(train_data2, vocab2, mincount = 1)
valid_data2 = remove_rare_words(valid_data2, vocab2, mincount = 1)
# train_data
print(train_data2[0])

remove rare words
['<s>', 'i', 'liked', 'your', 'idea', 'and', 'adopted', 'it', '.', '</s>']


In [16]:
# build n_grams
n=2
print("build ngram model with n = ", n)
model2 = build_ngram(train_data2, n)

build ngram model with n =  2


In [17]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model2, valid_data2, n=2))

The perplexity is 57.223483144798955


We notice that when using a larger training set, the perplexity reduced from 68.73 to 57.22. This means that the model becomes better when trying to predict a word given a context. This can be due to the fact that when training the model with more words the model has more information about the word.

## Change the value of n to 4

In [208]:
# build n_grams
n=4
print("build ngram model with n = ", n)
model2 = build_ngram(train_data2, n)

build ngram model with n =  4


In [209]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model2, valid_data2, n=4))

The perplexity is 53.58635810254969


Here we observe that increasing the size of the past n from 2 to 4 reduce the perplexity which means a better predictions.